In [2]:
import cv2
import pandas
import numpy as np

2.1

In [3]:
for i in range(300, 1100, 2): 
    I = cv2.imread('pedestrian/input/in%06d.jpg' % i) 
    cv2.imshow("I",I) 
    cv2.waitKey(10)
cv2.destroyAllWindows()

2.2

In [4]:
previous = 0

for i in range (300 ,1100):
    I = cv2.imread('pedestrian/input/in%06d.jpg' % i)
    I_gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    I_gray = I_gray.astype('uint8')
    current = I_gray
    frame = cv2.absdiff(current, previous)

    _, binary = cv2.threshold(frame, 25, 255, cv2.THRESH_BINARY)

    median = cv2.medianBlur(binary, 3)
    eroded = cv2.erode(median, np.ones((3,3), np.uint8))
    dilated = cv2.dilate(eroded, np.ones((15,15), np.uint8))

    cv2.imshow("motion detection", dilated)
    cv2.waitKey (10)
    previous = current
cv2.destroyAllWindows()



2.3

In [5]:
previous = 0

for i in range(300, 1100):
    I = cv2.imread('pedestrian/input/in%06d.jpg' % i)
    I_gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    I_gray = I_gray.astype('uint8')
    current = I_gray
    frame = cv2.absdiff(current, previous)

    _, binary = cv2.threshold(frame, 25, 255, cv2.THRESH_BINARY)

    median = cv2.medianBlur(binary, 3)
    eroded = cv2.erode(median, np.ones((3,3), np.uint8))
    dilated = cv2.dilate(eroded, np.ones((15,15), np.uint8))
    
    retval, labels, stats, centroids = cv2.connectedComponentsWithStats(dilated)
    I_VIS = current.copy().astype('uint8')
    if (stats.shape[0] > 1):
        tab = stats[1:,4]
        pi = np.argmax(tab)
        pi += 1
        cv2.rectangle(I_VIS,(stats[pi,0],stats[pi,1]),(stats[pi,0]+stats[pi,2],stats[pi ,1]+stats[pi,3]),(0,255,0),2)
        cv2.putText(I_VIS,"%f" % stats[pi,4],(stats[pi,0],stats[pi,1]),cv2. FONT_HERSHEY_SIMPLEX ,0.5,(0,255,0)) 
    cv2.imshow("Rect", I_VIS)
    
    previous = current.copy()
    
    if cv2.waitKey(10) == 27:
        break

cv2.destroyAllWindows()

2.4

In [ ]:
I1 = cv2.imread('pedestrian/input/in%06d.jpg' % 300)
I1 = cv2.cvtColor(I1, cv2.COLOR_BGR2GRAY)
I1 = I1.astype('int')

TP = 0
TN = 0
FP = 0
FN = 0

for i in range(301, 1100):
    I2 = cv2.imread('pedestrian/input/in%06d.jpg' % i)
    I2 = cv2.cvtColor(I2, cv2.COLOR_BGR2GRAY)
    I2 = I2.astype('int')

    Diff = abs(I2 - I1).astype('uint8')
    _, binary = cv2.threshold(Diff, 10, 255, cv2.THRESH_BINARY)

    median = cv2.medianBlur(binary, 7)
    eroded = cv2.erode(median, np.ones((3,3), np.uint8))
    dilated = cv2.dilate(eroded, np.ones((3,3), np.uint8))

    grountruth = cv2.imread('pedestrian/groundtruth/gt%06d.png' % i, cv2.IMREAD_GRAYSCALE)

    TP_matrix = np.logical_and((dilated > 0), (grountruth > 0))
    TN_matrix = np.logical_and((dilated == 0), (grountruth == 0))
    FP_matrix = np.logical_and((dilated > 0), (grountruth == 0))
    FN_matrix = np.logical_and((dilated == 0), (grountruth > 0))

    TP += TP_matrix.sum()
    TN += TN_matrix.sum()
    FP += FP_matrix.sum()
    FN += FN_matrix.sum()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

print('Precision:', P)
print('Recall:', R)
print('F1:', F1)

Precision: 0.3464534365046721
Recall: 0.9853834291375827
F1: 0.5126595968416064
